In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, countDistinct, approx_count_distinct, first, last, max, min, sum, sumDistinct, avg, mean, collect_list, collect_set, expr, stddev_pop


spark = SparkSession.builder.appName("chapter7").getOrCreate()

df = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("retail-data/all/*.csv").coalesce(5)
df.cache()
df.createOrReplaceTempView("dfTable")

df.count()

df.select(count("StockCode")).show(1, False)

df.select(countDistinct("StockCode")).show(1, False)

df.select(approx_count_distinct("StockCode", 0.1)).show(1, False)

df.select(first("StockCode"), last("StockCode")).show(1, False)

df.select(max("Quantity"), min("Quantity")).show(1, False)

df.select(sum("Quantity"), sumDistinct("Quantity")).show(1, False)

df.select(sum("Quantity").alias("total_sum"), count("Quantity").alias("number_of_quantities"), avg("Quantity").alias("avg_quantity"), mean("Quantity").alias("mean")).selectExpr("total_sum/number_of_quantities as avg", "avg_quantity", "mean").show(10, False)

# df.select(collect_list("Country"), collect_set("Country")).show(1, False)

df.groupBy("CustomerID", "InvoiceNo").count().show()

df.groupBy("InvoiceNo").agg(count("Quantity").alias("count_of_quantity"), expr("count(Quantity)")).show(1, False)

df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"), expr("stddev_pop(Quantity)")).show(1, False)

+----------------+
|count(StockCode)|
+----------------+
|541909          |
+----------------+

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|4070                     |
+-------------------------+

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|3364                            |
+--------------------------------+

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|85123A          |22138          |
+----------------+---------------+

+-------------+-------------+
|max(Quantity)|min(Quantity)|
+-------------+-------------+
|80995        |-80995       |
+-------------+-------------+

+-------------+----------------------+
|sum(Quantity)|sum(DISTINCT Quantity)|
+-------------+----------------------+
|5176450      |29310                 |
+-------------+----------------------+

+----------------+----------------+----------------+
|avg       

In [68]:
from pyspark.sql.functions import col, to_date, desc,dense_rank, rank
from pyspark.sql.window import Window

dfWithDate = df.withColumn("date", to_date(col("InvoiceDate")))
dfWithDate.createOrReplaceTempView("dfWithDate")

windowSpec = Window\
  .partitionBy("CustomerId", "date")\
  .orderBy(desc("Quantity"))\
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)
maxPurchase = max("Quantity").over(windowSpec)

purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
  .select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantityRank"),
    purchaseDenseRank.alias("quantityDenseRank"),
    maxPurchase.alias("maxPurchaseQuantity")).show()


+----------+----+--------+------------+-----------------+-------------------+
|CustomerId|date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----+--------+------------+-----------------+-------------------+
|     12346|null|   74215|           1|                1|              74215|
|     12346|null|  -74215|           2|                2|              74215|
|     12347|null|     240|           1|                1|                240|
|     12347|null|      48|           2|                2|                240|
|     12347|null|      36|           3|                3|                240|
|     12347|null|      36|           3|                3|                240|
|     12347|null|      36|           3|                3|                240|
|     12347|null|      36|           3|                3|                240|
|     12347|null|      36|           3|                3|                240|
|     12347|null|      36|           3|                3|       

In [93]:
from pyspark.sql.functions import grouping_id

dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

spark.sql("SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode)) ORDER BY CustomerId DESC, stockCode DESC").show(10, False)

spark.sql("SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode),()) ORDER BY CustomerId DESC, stockCode DESC").show(10, False)

rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date")
rolledUpDF.where(expr("Country is NULL")).show()
rolledUpDF.where("Date is NULL").show()
# rolledUpDF.show()


dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
  .select("Date", "Country", "sum(Quantity)").orderBy("Date").where("Country is NULL").show()

dfNoNull.cube("Date", "Country").agg(grouping_id(), sum(col("Quantity")))\
  .select("Date", "Country", "sum(Quantity)").orderBy("Date").where("Country is NULL").show()

dfNoNull.cube("customerId", "stockCode").agg(grouping_id(), sum("Quantity"))\
.orderBy(expr("grouping_id()"))\
.show()

pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

# pivoted.where("date > '2011-12-05'").select("date" ,"`USA_sum(CusomterID)`").show()



+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|18287     |85173    |48           |
|18287     |85040A   |48           |
|18287     |85039B   |120          |
|18287     |85039A   |96           |
|18287     |84920    |4            |
|18287     |84584    |6            |
|18287     |84507C   |6            |
|18287     |72351B   |24           |
|18287     |72351A   |24           |
|18287     |72349B   |60           |
+----------+---------+-------------+
only showing top 10 rows

+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|18287     |85173    |48           |
|18287     |85040A   |48           |
|18287     |85039B   |120          |
|18287     |85039A   |96           |
|18287     |84920    |4            |
|18287     |84584    |6            |
|18287     |84507C   |6            |
|18287     |72351B   |24           |
|18287     |72351A   |24           |
|18287     |